0. Imports

In [1]:
import pandas as pd
import datetime
import re

1. Load csv

In [2]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/datovy_súbor/log_jeden_den.csv', ';', usecols=['ip','url', 'unixtime'])
df.head(10)

,ip,url,unixtime
0,10.160.0.12,GET / HTTP/1.1,1582096524
1,10.160.0.126,GET / HTTP/1.1,1582119924
2,10.160.0.127,GET / HTTP/1.1,1582113753
3,10.160.0.13,GET / HTTP/1.1,1582105765
4,10.160.0.166,GET / HTTP/1.1,1582101139
5,10.160.0.166,GET / HTTP/1.1,1582102809
6,10.160.0.174,GET / HTTP/1.1,1582093467
7,10.160.0.177,GET / HTTP/1.1,1582101744
8,10.160.0.177,GET /univerzita/kontakt/adresar-osob HTTP/1.1,1582104899
9,10.160.0.177,GET / HTTP/1.1,1582117002


3. Add new columns with default values of 0



In [3]:
df['INTERNY'] = 0
df['STUD'] = 0
df['ZAM'] = 0
df['PON'] = 0
df['UTO'] = 0
df['STR'] = 0
df['STV'] = 0
df['PIA'] = 0
df['SOB'] = 0
df['NED'] = 0

4. Whether it is access from inside the network or outside the network based on IP 



*   10.160.0.xxx -> IN/STUDENT
*   10.160.1.xxx -> IN/STUDENT
*   10.160.2.0xx -> IN/STUDENT
*   10.160.2.1xx -> IN/STUDENT
*   10.160.2.2xx -> IN/ZAMEST
*   10.160.3.xxx -> IN/STUDENT
*   10.160.xxx.xxx -> IN/ZAMEST
*   other -> OUT



In [4]:
def classify_ips(ip, index):
    if(ip.__contains__('10.160.0')):
        df.loc[index, 'STUD'] = 1
        df.loc[index, 'INTERNY'] = 1
    elif(ip.__contains__('10.160.1')):
        df.loc[index, 'STUD'] = 1
        df.loc[index, 'INTERNY'] = 1
    elif(ip.__contains__('10.160.2.0')):
        df.loc[index, 'STUD'] = 1
        df.loc[index, 'INTERNY'] = 1
    elif(ip.__contains__('10.160.2.1')):
        df.loc[index, 'STUD'] = 1
        df.loc[index, 'INTERNY'] = 1
    elif(ip.__contains__('10.160.2.2')):
        df.loc[index, 'ZAM'] = 1
        df.loc[index, 'INTERNY'] = 1
    elif(ip.__contains__('10.160.3')):
        df.loc[index, 'STUD'] = 1
        df.loc[index, 'INTERNY'] = 1
    elif(ip.__contains__('10.160')):
        df.loc[index, 'ZAM'] = 1
        df.loc[index, 'INTERNY'] = 1

5. Information on the weekday of access 

In [5]:
def classify_weekday(weekday, index):
  if (weekday == 0):
    df.loc[index, 'PON'] = 1
  elif (weekday == 1):
    df.loc[index, 'UTO'] = 1
  elif (weekday == 2):
    df.loc[index, 'STR'] = 1
  elif (weekday == 3):
    df.loc[index, 'STV'] = 1
  elif (weekday == 4):
    df.loc[index, 'PIA'] = 1
  elif (weekday == 5):
    df.loc[index, 'SOB'] = 1
  else:
    df.loc[index, 'NED'] = 1

6. Cycle through dataframe and apply all changes

In [6]:
# Cycle through dataframe
for index, row in df.iterrows():

    # Create regex for url to get category name
    req = re.search("GET /(.+?)/",row['url'])
    if req:
      value = req.group(1)

      # Replace all HTTP with 'úvod'
      if (value == ' HTTP'):
        value = 'uvod' 
      df.loc[index, 'url'] = value

    # Classify ip
    classify_ips(row['ip'], index)

    # Get a hour of access and hour stv
    hod = datetime.datetime.fromtimestamp(int(row['unixtime'])).strftime('%H')
    df.loc[index, 'HOD'] = hod
    df.loc[index, 'HOD_STV'] = int(hod) * int(hod)

    # Get a weekday of access
    classify_weekday(datetime.datetime.fromtimestamp(int(row['unixtime'])).weekday(), index) 

df.head(10)

,ip,url,unixtime,INTERNY,STUD,ZAM,PON,UTO,STR,STV,PIA,SOB,NED,HOD,HOD_STV
0,10.160.0.12,uvod,1582096524,1,1,0,0,0,1,0,0,0,0,07,49.0
1,10.160.0.126,uvod,1582119924,1,1,0,0,0,1,0,0,0,0,13,169.0
2,10.160.0.127,uvod,1582113753,1,1,0,0,0,1,0,0,0,0,12,144.0
3,10.160.0.13,uvod,1582105765,1,1,0,0,0,1,0,0,0,0,09,81.0
4,10.160.0.166,uvod,1582101139,1,1,0,0,0,1,0,0,0,0,08,64.0
5,10.160.0.166,uvod,1582102809,1,1,0,0,0,1,0,0,0,0,09,81.0
6,10.160.0.174,uvod,1582093467,1,1,0,0,0,1,0,0,0,0,06,36.0
7,10.160.0.177,uvod,1582101744,1,1,0,0,0,1,0,0,0,0,08,64.0
8,10.160.0.177,univerzita,1582104899,1,1,0,0,0,1,0,0,0,0,09,81.0
9,10.160.0.177,uvod,1582117002,1,1,0,0,0,1,0,0,0,0,12,144.0


7. Delete unnecessary columns

In [7]:
df.pop('ip')
df.pop('unixtime')

0        1582096524
1        1582119924
2        1582113753
3        1582105765
4        1582101139
            ...    
20643    1582144086
20644    1582162245
20645    1582132345
20646    1582132345
20647    1582147423
Name: unixtime, Length: 20648, dtype: int64

8. Save to csv

In [8]:
df.to_csv('Laca_datovy_subor.csv', sep=';')